In [1]:
# fundemental modules
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import time
import pprint
from tqdm.notebook import trange

from ddqn import DDQNAgent # type: ignore
from utils import load_config
from obs import observation_shape

aggregate_stats_every=100
best_score = -1000.0
score_history = []
episode_lens = []
avg_history = []
std_history = []
avg_history_100 = []



2023-12-14 10:12:45.020832: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-14 10:12:45.302644: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 10:12:45.302709: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 10:12:45.303977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 10:12:45.408150: I tensorflow/core/platform/cpu_feature_g

In [8]:
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure({   "action": {"lateral": True,
            "longitudinal": True,
            "type": "ContinuousAction"},
            "observation": {
                "type": "GrayscaleObservation",
                "observation_shape": (64, 32),
                "stack_size": 2,
                "weights": [0.2989, 0.5870, 0.1140],
                "scaling": 1.75},
    "show_trajectories": True,
    "duration": 80,
    "collision_reward": -1,
    "simulation_frequency": 15,
    "policy_frequency": 5,
    "real_time_rendering": True
}) # type: ignore
(obs, info), done = env.reset(), False
input = observation_shape(obs) 


/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: uint8
  logger.warn(
/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [3]:
agent = DDQNAgent(alpha=0.001, gamma=0.9, epsilon=1.0, obs_shape=input.shape,
                  batch_size=64, epsilon_dec=0.999, epsilon_end=0.05, mem_size=100000,
                  min_mem_size=100, replace_target=100, learning_rate=0.0003)


# agent = DDQNAgent(alpha=0.001, gamma=0.9, epsilon=1.0, obs_shape=input.shape,
#                   batch_size=64, epsilon_dec=0.9995, epsilon_end=0.05, mem_size=100000,
#                   min_mem_size=100, replace_target=1000, learning_rate=0.0003)
print("Agent is initialized.")


Agent is initialized.


2023-12-14 10:12:50.616366: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 10:12:50.731368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 10:12:50.731742: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# main loop
for episode in trange(500, desc='Test episodes'):

        (observation, info), done = env.reset(), False
        truncated = False
        observation = observation_shape(observation) 

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            
            action, action_index = agent.get_action(observation, deterministic=False)
            new_observation, reward, done, truncated, new_info = env.step(action=action)
            new_observation = observation_shape(new_observation)

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >= 200:
                truncated = True
            
            agent.remember(state=observation, action=action_index, done=done,
                            reward=reward, new_state=new_observation)
            agent.train()

            observation = new_observation

        episode_lens.append(episode_len)

        score_history.append(episode_reward)
        avg_score = np.mean(score_history)
        avg_history.append(avg_score)
        std_score = np.std(score_history)
        std_history.append(std_score)

        avg_score_100 = np.mean(score_history[-100:])
        avg_history_100.append(avg_score_100)

        if agent.memory.mem_cntr > 100:
            best_score = avg_score
            agent.save_model(episode)

        agent.tensorboard.update_stats(episode_rew = episode_reward,
                                       average_rew =avg_score,
                                       average_100_reward = avg_score_100,
                                       std_rew=std_score,
                                       epsilon=agent.epsilon,
                                       episode_len = episode_len)
        

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len,
              'avg score %.1f' % avg_score, 'avg_score_100 %.1f' %avg_score_100,'std score %.1f' % std_score)

Test episodes:   0%|          | 0/500 [00:00<?, ?it/s]

/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: uint8
  logger.warn(
/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2023-12-14 10:12:59.793138: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


episode  0 score 0.6 ep len 6 avg score 0.6 avg_score_100 0.6 std score 0.0
episode  1 score 1.8 ep len 16 avg score 1.2 avg_score_100 1.2 std score 0.6
episode  2 score 0.0 ep len 7 avg score 0.8 avg_score_100 0.8 std score 0.7
episode  3 score 0.9 ep len 9 avg score 0.8 avg_score_100 0.8 std score 0.6
episode  4 score -2.9 ep len 25 avg score 0.1 avg_score_100 0.1 std score 1.6
episode  5 score 0.0 ep len 10 avg score 0.1 avg_score_100 0.1 std score 1.4
episode  6 score 0.2 ep len 8 avg score 0.1 avg_score_100 0.1 std score 1.3


2023-12-14 10:13:01.896391: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-12-14 10:13:02.785694: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-14 10:13:04.447669: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f53b816cbd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-14 10:13:04.447681: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-12-14 10:13:04.452832: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-14 10:13:04.518882: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Target Updated
-----saving models------
episode  7 score -3.6 ep len 23 avg score -0.4 avg_score_100 -0.4 std score 1.7
-----saving models------
episode  8 score -0.1 ep len 10 avg score -0.3 avg_score_100 -0.3 std score 1.6
-----saving models------
episode  9 score -3.1 ep len 29 avg score -0.6 avg_score_100 -0.6 std score 1.8
-----saving models------
episode  10 score -1.1 ep len 20 avg score -0.7 avg_score_100 -0.7 std score 1.7
-----saving models------
episode  11 score 1.1 ep len 5 avg score -0.5 avg_score_100 -0.5 std score 1.7
-----saving models------
episode  12 score 0.7 ep len 5 avg score -0.4 avg_score_100 -0.4 std score 1.6
-----saving models------
episode  13 score 1.0 ep len 5 avg score -0.3 avg_score_100 -0.3 std score 1.6
-----saving models------
episode  14 score 0.0 ep len 3 avg score -0.3 avg_score_100 -0.3 std score 1.6
-----saving models------
episode  15 score -0.7 ep len 13 avg score -0.3 avg_score_100 -0.3 std score 1.5
Target Updated
-----saving models------
ep

In [5]:
n_steps = sum(episode_lens)
print(n_steps)


17858


In [9]:
# env = record_videos(env)
from gymnasium.wrappers import RecordVideo

# data = load_config()
# env = gym.make('racetrack-v0', render_mode = 'rgb_array')
# env.configure(data) # type: ignore

env = RecordVideo(env, video_folder="videos", episode_trigger=lambda e: True)
env.unwrapped.set_record_video_wrapper(env)

# agent.load_model()
# main loop
for episode in trange(3, desc='Test episodes'):

        (observation, info), done = env.reset(), False
        truncated = False
        observation = observation_shape(observation) 

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            
            action, action_index = agent.get_action(observation, deterministic=True)
            new_observation, reward, done, truncated, new_info = env.step(action=action)
            new_observation = observation_shape(new_observation)

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >= 200:
                truncated = True
            
            agent.remember(state=observation, action=action_index, done=done,
                            reward=reward, new_state=new_observation)
            agent.train()

            observation = new_observation

env.close()     

/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/o/Documents/thesis/two_acts_cnn/ddqn/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: uint8
  logger.warn(
/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/wrappers/monitoring/video_recorder.py:178: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")


Target Updated
Moviepy - Building video /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-0.mp4
Target Updated
Moviepy - Building video /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-1.mp4
Target Updated
Moviepy - Building video /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-2.mp4.
Moviepy - Writing video /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /home/o/Documents/thesis/two_acts_cnn/ddqn/videos/rl-video-episode-2.mp4
